In [6]:
import requests

API_KEY = '413d101d1e60dd334ffe8202781937d45e25044e9defea1db882eec3962bb31f'  
BASE_URL = 'https://apiv2.allsportsapi.com/football/'

def get_country_key(country_name):
    params = {
        'met': 'Countries',
        'APIkey': API_KEY
    }
    response = requests.get(BASE_URL, params=params)
    data = response.json()
    for country in data['result']:
        if country['country_name'].lower() == country_name.lower():
            return country['country_key']
    return None

country_key = get_country_key('Spain')
print(f"Country Key for Spain: {country_key}")



Country Key for Spain: 6


In [5]:
import requests
import json

BASE_URL = "https://apiv2.allsportsapi.com/football/"
API_KEY = '413d101d1e60dd334ffe8202781937d45e25044e9defea1db882eec3962bb31f'  

def get_country_key(country_name):
    params = {
        'met': 'Countries',
        'APIkey': API_KEY
    }
    response = requests.get(BASE_URL, params=params)
    data = response.json()

    for country in data.get('result', []):
        if country.get('country_name', '').lower() == country_name.lower():
            return country.get('country_key')
    return None

def get_league_id(country_key, league_name):
    params = {
        'met': 'Leagues',
        'APIkey': API_KEY,
        'countryId': country_key
    }
    response = requests.get(BASE_URL, params=params)
    data = response.json()


    for league in data.get('result', []):
        if league.get('league_name', '').lower() == league_name.lower():
            return league.get('league_id') or league.get('league_key') 

    print(f"League '{league_name}' not found for country key {country_key}.")
    return None

country_key = get_country_key('Spain')
if country_key:
    league_id = get_league_id(country_key, 'La Liga')  
    if league_id:
        print(f"League ID for La Liga: {league_id}")
    else:
        print("Could not find the League ID.")
else:
    print("Could not find the Country Key.")


League ID for La Liga: 302


In [7]:
def get_standings(league_id):
    params = {
        'met': 'Standings',
        'APIkey': API_KEY,
        'leagueId': league_id
    }
    response = requests.get(BASE_URL, params=params)
    data = response.json()
    return data['result']['total']

standings = get_standings(league_id)


In [8]:
import pandas as pd

def create_standings_df(standings):
    columns = [
        'standing_place', 'standing_team', 'standing_P', 'standing_W',
        'standing_D', 'standing_L', 'standing_F', 'standing_A',
        'standing_GD', 'standing_PTS', 'team_key', 'league_key',
        'league_season', 'league_round', 'standing_updated'
    ]
    df = pd.DataFrame(standings)
    df = df[columns]
    return df

standings_df = create_standings_df(standings)
print(standings_df.head())


   standing_place standing_team  standing_P  standing_W  standing_D  \
0               1     Barcelona          34          25           4   
1               2   Real Madrid          34          23           6   
2               3   Atl. Madrid          34          19          10   
3               4    Ath Bilbao          34          16          13   
4               5    Villarreal          34          16          10   

   standing_L  standing_F  standing_A  standing_GD  standing_PTS  team_key  \
0           5          91          33           58            79        97   
1           5          69          33           36            75        76   
2           5          56          27           29            67        73   
3           5          50          26           24            61      7258   
4           8          60          47           13            58       162   

   league_key league_season league_round     standing_updated  
0         302     2024/2025             

In [9]:
#standings_df.to_excel('la_liga_standings.xlsx', index=False)


In [10]:
def analyze_standings(df):
    df['standing_place'] = df['standing_place'].astype(int)
    df_sorted = df.sort_values('standing_place')

    analysis = []

    for index, row in df_sorted.iterrows():
        position = row['standing_place']
        team = row['standing_team']
        if position <= 4:
            qualification = 'Champions League'
        elif position == 5:
            qualification = 'Europa League Group Stage'
        elif position == 6:
            qualification = 'Europa Conference League Qualifiers'
        elif position >= len(df_sorted) - 2:
            qualification = 'Relegation'
        else:
            qualification = 'None'
        analysis.append({
            'Team': team,
            'Position': position,
            'Qualification': qualification
        })

    analysis_df = pd.DataFrame(analysis)
    return analysis_df

analysis_df = analyze_standings(standings_df)
print(analysis_df)


              Team  Position                        Qualification
0        Barcelona         1                     Champions League
1      Real Madrid         2                     Champions League
2      Atl. Madrid         3                     Champions League
3       Ath Bilbao         4                     Champions League
4       Villarreal         5            Europa League Group Stage
5            Betis         6  Europa Conference League Qualifiers
6       Celta Vigo         7                                 None
7   Rayo Vallecano         8                                 None
8          Osasuna         9                                 None
9         Mallorca        10                                 None
10   Real Sociedad        11                                 None
11        Valencia        12                                 None
12          Getafe        13                                 None
13        Espanyol        14                                 None
14        

In [11]:
# import streamlit as st
# import pandas as pd

# st.title("Football League Standings Analysis - La Liga")
# st.markdown("This website is made with Streamlit. It explains the coding process, algorithm analysis, and the final result table. - Developed by Hritesh Ghosh")

# st.header("Author & Task")
# st.markdown("""
# **Author:** Hritesh Ghosh  
# **Task:** Football League Standings Analysis (Spanish La Liga)
# """)

# st.header("Overview")
# st.markdown("""
# This script connects to the AllSportsAPI to fetch live football standings for Spain's La Liga. It then processes the data and categorizes teams based on their positions for various European competitions and relegation.
# """)

# st.header("Steps Performed")
# steps = [
#     "1. Authenticated the API using a free key from AllSportsAPI.",
#     "2. Retrieved the country_key for Spain.",
#     "3. Fetched the league_id for Spanish La Liga.",
#     "4. Pulled current league standings using the API.",
#     "5. Converted API data into a pandas DataFrame with selected columns.",
#     "6. Saved the DataFrame to an Excel file named `la_liga_standings.xlsx`.",
#     "7. Created a function to analyze standings and return:",
#     "   - Top 4 teams → Champions League (Group Stage)",
#     "   - 5th team → Europa League (Group Stage)",
#     "   - 6th team → Europa Conference League (Qualifiers)",
#     "   - Bottom 3 teams → Relegation",
#     "8. Output the results in a structured DataFrame."
# ]
# for step in steps:
#     st.markdown(f"- {step}")

# st.header("Analysis Algorithm")
# st.markdown("""
# **Input:**  
# A Pandas DataFrame `df` containing La Liga standings with at least the columns `standing_place` and `standing_team`.

# **Output:**  
# A new DataFrame containing only the teams that qualify for European competitions or face relegation, along with their positions and qualification labels.

# **Algorithm Steps:**
# 1. Convert the `standing_place` column in `df` to integer type.  
# 2. Sort the DataFrame `df` in ascending order by `standing_place`.  
# 3. Initialize an empty list `analysis`.  
# 4. For each row in the sorted DataFrame:  
#    - Let `pos` be the value of `standing_place`.  
#    - Let `team` be the value of `standing_team`.  
#    - If `pos` is between 1 and 4 (inclusive), set `tag` to `"Champions League"`.  
#    - Else if `pos` is equal to 5, set `tag` to `"Europa League Group Stage"`.  
#    - Else if `pos` is equal to 6, set `tag` to `"Europa Conference League Qualifiers"`.  
#    - Else if `pos` is greater than total number of teams - 3, set `tag` to `"Relegation"`.  
#    - Otherwise, skip the team.  
#    - Append a dictionary `{ 'Team': team, 'Position': pos, 'Qualification': tag }` to `analysis`.  
# 5. Convert the `analysis` list into a new DataFrame and return it.
# """)

# st.header("Result: La Liga Qualification Summary")

# data = {
#     "Team": [
#         "Barcelona", "Real Madrid", "Atl. Madrid", "Ath Bilbao",
#         "Villarreal", "Betis", "Las Palmas", "Leganes", "Valladolid"
#     ],
#     "Position": [1, 2, 3, 4, 5, 6, 18, 19, 20],
#     "Qualification": [
#         "Champions League", "Champions League", "Champions League", "Champions League",
#         "Europa League Group Stage", "Europa Conference League Qualifiers",
#         "Relegation", "Relegation", "Relegation"
#     ]
# }

# df = pd.DataFrame(data)
# st.dataframe(df, use_container_width=True)


In [12]:
import webbrowser; webbrowser.open_new_tab('https://baarproject.streamlit.app/')

True